In [ ]:
from scapy.all import *

In [ ]:
def keep_sniffing(pckt):
    return False
def add_network(pckt, known_networks):
    if not pckt.haslayer(Dot11):
        print(pckt.summary())
    try:
        bssid = pckt[Dot11].addr3
        channel = int(ord(pckt[Dot11Elt:3].info))
        if bssid not in known_networks:
            known_networks[bssid] = (channel,pckt)
            print(bssid+" "+str(channel))
    except IndexError:
        pckt.summary()
    except TypeError:
        pckt.summary()
    
network = {}

In [ ]:
s = sniff(iface='mon0', lfilter=lambda x: (x.haslayer(Dot11Beacon) or x.haslayer(Dot11ProbeResp)), stop_filter=keep_sniffing, prn=lambda x:add_network(x, network))

In [ ]:
i = 0
for item in network:
    i = i+1
    print(i, item, network[item][1].summary())

In [ ]:
network['dc:51:eb:0f:0e:fd'][1].show()